# Intro

Hi guys, I am ML novice and I would like to first give many thanks to kagglers who share their awesome notebooks. They help me learn a lot!

This notebook is credit to a famous public pytoch resNet Notebook shared by Lin.   
https://www.kaggle.com/a763337092/blending-tensorflow-and-pytorch  
https://www.kaggle.com/a763337092/neural-network-starter-pytorch-version  
https://www.kaggle.com/a763337092/pytorch-resnet-starter-training  

I am more familiar with keras, so I replicated it in Keras to further tuning the model. I share the model with you guys for your own testing. 


# My questions
For me, I met these problems while tuning the model, maybe someone can help to advice? Thanks -

(1) The performance of this keras model got similar performance as the pytorch model in the following link. But it is worse than the submitted model shared by Lin.   
https://www.kaggle.com/a763337092/pytorch-resnet-starter-training  

-> Can anyone advice how should I change parameters to train the model better? I changed the EARLYSTOP_NUM to 20, the performance improves around 20%, but still not as good as the public one.

(2) In the public model, Lin blends 5 pytorch resNet model and one tf model. But when I submit the keras resNet model, 2 resNet models is the maximum even after I speed up other process. 
https://www.kaggle.com/a763337092/blending-tensorflow-and-pytorch  

-> Is it because Keras model naturally runs slower than pytorch model? Is there any method to speed up the inference?

Thanks!


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
import json
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
import pickle
import keras


#%% - load data - 
train_pickle_file = '../input/pickling/train.csv.pandas.pickle'
train = pickle.load(open(train_pickle_file, 'rb'))

features = [c for c in train.columns if "feature" in c]
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [ ]:
#%% - build model - 

def create_resnet(n_features, n_labels, learning_rate, label_smoothing, adam_decay):
    inp = Input(shape=(n_features,), name='Input')
    dropout_rate = 0.2
    hidden_size = 256
    
    x = BatchNormalization()(inp)
    x = Dropout(dropout_rate)(x)
    
    head_1 = tf.keras.Sequential([
        Dense(hidden_size),
        BatchNormalization(),
        tf.keras.layers.LeakyReLU(0.01),
        #tf.keras.layers.PReLU(),
        Dropout(dropout_rate)],
        name='Head1')
    
    x1 = head_1(x)
    
    x = Concatenate()([x,x1])
    
    head_2 = tf.keras.Sequential([
        Dense(hidden_size),
        BatchNormalization(),
        tf.keras.layers.LeakyReLU(0.01),
        #tf.keras.layers.PReLU(),
        Dropout(dropout_rate),
    ], name='Head2')
    
    x2 = head_2(x)
    
    x = Concatenate()([x1, x2])
    
    head_3 = tf.keras.Sequential([
        Dense(hidden_size),
        BatchNormalization(),
        tf.keras.layers.LeakyReLU(0.01),
        #tf.keras.layers.PReLU(),
        Dropout(dropout_rate)
    ], name='Head3')
    
    x3 = head_3(x)
    x = Concatenate()([x2, x3])
    
    head_4 = tf.keras.Sequential([
        Dense(hidden_size),
        BatchNormalization(),
        tf.keras.layers.LeakyReLU(0.01),
        #tf.keras.layers.PReLU(),
        Dropout(dropout_rate),
    ], name='Head4')
    x4 = head_4(x)
    x = Concatenate()([x3, x4])
    
    output = Dense(n_labels)(x)
    output = tf.keras.layers.Activation("sigmoid")(output)
    model = tf.keras.models.Model(inputs=inp, outputs=output)
    
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=adam_decay),
                   loss=tf.keras.losses.BinaryCrossentropy(
                       label_smoothing=label_smoothing),
                   metrics=[tf.keras.metrics.AUC(name="auc"), 'accuracy']
                   )
    #model.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
    #              loss=tf.keras.losses.BinaryCrossentropy(
    #                  label_smoothing=label_smoothing),
    #              metrics=[tf.keras.metrics.AUC(name="auc"),'accuracy']
    #              )
    return model

In [ ]:
#%% - train test data split - 

train = train.query('date > 85').reset_index(drop=True)

# without drop weight==0
train.fillna(train.mean(), inplace=True)

#f_mean = np.mean(train[features[1:]].values,axis=0)
f_mean = train.mean()

train['action'] = (train['resp'] > 0).astype('int')
train['action_1'] = (train['resp_1'] > 0).astype('int')
train['action_2'] = (train['resp_2'] > 0).astype('int')
train['action_3'] = (train['resp_3'] > 0).astype('int')
train['action_4'] = (train['resp_4'] > 0).astype('int')

valid = train.loc[(train.date >= 450) & (
    train.date < 500)].reset_index(drop=True)
train = train.loc[train.date < 450].reset_index(drop=True)

target_cols = ['action', 'action_1', 'action_2', 'action_3', 'action_4']

feat_cols = [f'feature_{i}' for i in range(130)]
all_feat_cols = [col for col in feat_cols]

train['cross_41_42_43'] = train['feature_41'] + \
    train['feature_42'] + train['feature_43']
train['cross_1_2'] = train['feature_1'] / (train['feature_2'] + 1e-5)
valid['cross_41_42_43'] = valid['feature_41'] + \
    valid['feature_42'] + valid['feature_43']
valid['cross_1_2'] = valid['feature_1'] / (valid['feature_2'] + 1e-5)

all_feat_cols.extend(['cross_41_42_43', 'cross_1_2'])

In [ ]:
#%% - define call back to evalute score on each epoch - 
class TrainingEva(Callback):
    def __init__(self, test, state_dir, fold):
        super(TrainingEva, self).__init__()
        # test = [X,resp,w,date]
        self.test_X = test[0]
        self.test_resp = test[1]
        self.test_w = test[2]
        self.test_date = test[3]
        self.th = 0.5
        self.log_dir = state_dir+f'/logs_{fold}.json'
        self.history = {'epoch': [], 'score': []}

    def score(self, date, weight, resp, action):
        count_i = len(np.unique(date))
        Pi = np.bincount(date, weight * resp * action)
        t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
        u = np.clip(t, 0, 6) * np.sum(Pi)
        return u

    def on_epoch_end(self, epoch, logs=None):
        test_pred = self.model.predict(self.test_X)[:, 0]
        test_action = np.where(test_pred >= self.th, 1, 0).astype(int)
        test_u_score = self.score(
            self.test_date, self.test_w, self.test_resp, test_action)

        self.history['epoch'].append(epoch)
        self.history['score'].append(test_u_score)

        print("fold:{} epoch:{} val_score:{}".format(fold, epoch, test_u_score))

        with open(self.log_dir, 'w') as f:
            json.dump(self.history, f)

In [ ]:
#%% - train model - 
SEED = 42
LABEL_SMOOTHING = 0.005
BATCH_SIZE = 8192
EPOCHS = 200
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
EARLYSTOP_NUM = 3
NFOLDS = 5

outpath = './'
TRAINING = True
if TRAINING:
    for fold in range(NFOLDS):
        #  - seed -
        tf.random.set_seed(SEED+fold)
        np.random.seed(SEED+fold)
        # 1. prepare data
        X_train = train[all_feat_cols].values
        y_train = train[target_cols].values

        X_valid = valid[all_feat_cols].values
        y_valid = valid[target_cols].values
        
        resp_valid = valid['resp'].values
        w_valid = valid['weight'].values
        date_valid = valid['date'].values

        # 2. create model
        tf.keras.backend.clear_session()
        clf = create_resnet(len(all_feat_cols), len(target_cols), LEARNING_RATE,
                            LABEL_SMOOTHING, WEIGHT_DECAY)

        # 3. create callback
        esCallBack = EarlyStopping(
            'val_auc', mode='max', patience=EARLYSTOP_NUM, restore_best_weights=(False))

        filepath = './'+outpath+'/weights-improvement-{epoch:02d}-{val_auc:.2f}' + \
            f'-fold:{fold}.hdf5'
        checkpoint = ModelCheckpoint(
            filepath, verbose=0, save_best_only=(True), mode='max')

        evaCallBack = TrainingEva([X_valid, resp_valid, w_valid, date_valid],
                                  './'+outpath, fold)

        # 4. train model
        history = clf.fit(X_train, y_train, validation_data=(X_valid, y_valid),
                          epochs=EPOCHS, batch_size=BATCH_SIZE,
                          callbacks=[esCallBack, checkpoint, evaCallBack])
        # 5. save log -
        auc = history.history['auc']
        val_auc = history.history['val_auc']
        loss_score = history.history['loss']
        val_loss_score = history.history['val_loss']
        history_df = pd.DataFrame(
            {'AUC': auc, 'val_AUC': val_auc, 'loss': loss_score, 'val_loss': val_loss_score})
        history_df.to_csv('./'+outpath+f'/history_df_{fold}.csv')

        # - save model -
        clf.save('./'+outpath+f'/model_final_{fold}.h5')

That's it!